In [1]:
import pickle
import os
import sys
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
with open('Data/papers.txt', 'r') as f:
    papers = f.readlines()

In [18]:
import pandas as pd

def load_and_process_data(file_path):
    papers = []
    
    with open(file_path, 'r') as f:
        paper = []
        for i, line in enumerate(f):
            if i == 0:
                continue  # Skip the first line
            if line == '\n':  # Blank line indicating end of one paper's data
                if paper:
                    paper_data = process_paper(paper)
                    papers.append(paper_data)
                    paper = []
            else:
                paper.append(line.strip())
        
        # Process the last paper if there's no newline at the end
        if paper:
            paper_data = process_paper(paper)
            papers.append(paper_data)
    
    return pd.DataFrame(papers)


def process_paper(paper):
    paper_data = {}
    
    title_line = next((line for line in paper if line.startswith('#*')), None)
    paper_data['title'] = title_line[2:].strip() if title_line else None
    
    
    # Extract authors
    authors_line = next((line for line in paper if line.startswith('#@')), None)
    if authors_line:
        authors = authors_line[2:].strip()
        paper_data['authors'] = authors.split(',')
    
    # Extract year
    year_line = next((line for line in paper if line.startswith('#t')), None)
    if year_line:
        paper_data['year'] = year_line[2:].strip()
    
    # Extract venue
    venue_line = next((line for line in paper if line.startswith('#c')), None)
    if venue_line:
        paper_data['venue'] = venue_line[2:].strip()
    
    # Extract paper index
    index_line = next((line for line in paper if line.startswith('#index')), None)
    if index_line:
        paper_data['index'] = index_line[6:].strip()
    
    # Extract citations (each citation on a separate line starting with '#%')
    citations = []
    for line in paper:
        if line.startswith('#%'):
            citation = line[2:].strip()
            citations.append(citation)
    paper_data['citations'] = citations if citations else []
    
    # Extract abstract
    abstract_line = next((line for line in paper if line.startswith('#!')), None)
    if abstract_line:
        paper_data['abstract'] = abstract_line[2:].strip()
    
    # Extract categories (if available)
    categories_line = next((line for line in paper if line.startswith('Values')), None)
    if categories_line:
        categories = categories_line[7:].strip()
        paper_data['categories'] = categories.split(',') if categories else []

    # Set pickle_id to paper index
    paper_data['pickle_id'] = paper_data['index']  # Set pickle ID to paper index
    return paper_data


# Example usage
df = load_and_process_data('Data/papers.txt')

In [19]:
df

,title,authors,year,venue,index,citations,pickle_id,abstract
0,Automated Deduction in Geometry: 5th Internati...,"[Hoon Hong, Dongming Wang]",2006,,0,[],0,NaN
1,A+ Certification Core Hardware (Text & Lab Man...,[Charles J. Brooks],2003,,1,[],1,NaN
2,Performance engineering in industry: current p...,"[Ahmed E. Hassan, Parminder Flora]",2007,Proceedings of the 6th international workshop ...,2,[],2,This panel session discusses performance engin...
3,"Dude, You Can Do It! How to Build a Sweeet PC","[Darrel Creacy, Carlito Vicencio]",2005,,3,[],3,Whether you're frustrated with current PC offe...
4,What Every Programmer Needs to Know about Secu...,"[Neil Daswani, Anita Kesavan]",2006,,4,[],4,NaN
...,...,...,...,...,...,...,...,...
629809,Mining A,[],2008,Proceedings of the VLDB Endowment,629809,[],629809,NaN
629810,Review article,[],2008,Communications of the ACM,629810,[],629810,NaN
629811,Multimodal system evaluation using modality ef...,"[Manolis Perakakis, Alexandros Potamianos]",2008,Proceedings of the 10th international conferen...,629811,"[294663, 302639, 572828]",629811,"In this paper, we propose two new objective me..."
629812,Computer System Architecture,[V. K. Jain],2007,,629812,[],629812,NaN


In [20]:
print(df.shape)

(629814, 8)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629814 entries, 0 to 629813
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      629814 non-null  object
 1   authors    629814 non-null  object
 2   year       629814 non-null  object
 3   venue      629814 non-null  object
 4   index      629814 non-null  object
 5   pickle_id  629814 non-null  object
 6   abstract   281080 non-null  object
dtypes: object(7)
memory usage: 33.6+ MB
